# Load packages

In [ ]:
import Astroshaper
import DataFrames
import GLMakie
import JLD2

import Revise
import Astroshaper
import SPICE
import Downloads
import Statistics

# Download files

In [ ]:
path_meta_new = "hyb2_v03.tm"
if !isfile(path_meta_new)
    # Dowonload files for SPICE
    cmd = `wget -nv -m -np -nH --cut-dirs=3 -R "index.html*" --execute robots=off --wait=1 https://data.darts.isas.jaxa.jp/pub/hayabusa2/spice_bundle/spice_kernels/`
    run(cmd)

    # Update path
    path_meta_old = "spice_kernels/mk/hyb2_v03.tm"
    path_meta_new = "hyb2_v03.tm"
    cp(path_meta_old, path_meta_new, force=true)
    script = read(path_meta_new, String)
    script = replace(script, "PATH_VALUES     = ( '..'      )"=>"PATH_VALUES     = ('$(abspath("spice_kernels"))')")
    write(path_meta_new, script)
end

path_obj = "SHAPE_SFM_49k_v20180804.obj"
if !isfile(path_obj)
    url_obj = "https://data.darts.isas.jaxa.jp/pub/hayabusa2/paper/Watanabe_2019/SHAPE_SFM_49k_v20180804.obj"
    Downloads.download(url_obj, path_obj)
end

# Load data with SPICE

In [ ]:
SPICE.furnsh(path_meta_new)
et_start = SPICE.utc2et("2018-07-01T00:00:00")
et_end   = SPICE.utc2et("2018-07-02T00:00:00")
step     = 76.3262  # Rotation of 1 deg
et_range = et_start : step : et_end
@show et_range
@show length(et_range)

# Indices of et_range to be saved.
# Save only the last rotation.
save_range = findall(et_range .> et_range[end] - 7.63262 * 3600)
@show save_range[begin]
@show save_range[end]
@show length(save_range);

# Sun's position in the RYUGU_FIXED frame
sun_ryugu = [SPICE.spkpos("SUN", et, "RYUGU_FIXED", "None", "RYUGU")[1]*1000 for et in et_range]
# Transformation matrix from RYUGU_FIXED to J2000
RYUGU_TO_J2000 = [SPICE.pxform("RYUGU_FIXED", "J2000", et) for et in et_range]
SPICE.kclear()

# Load obj file

In [ ]:
path_jld = splitext(path_obj)[1]*".jld2"
if isfile(path_jld)
    shape = Astroshaper.ShapeModel(path_jld; scale=1000, find_visible_facets=true, save_shape=true)
else
    shape = Astroshaper.ShapeModel(path_obj; scale=1000, find_visible_facets=true, save_shape=true)
end

# TPM

In [ ]:
thermo_params = Astroshaper.ThermoParams(
    A_B   = 0.04,  # Bolometric Bond albedo
    A_TH  = 0.0,
    k     = 0.1,
    ρ     = 1270.0,
    Cp    = 600.0,
    ϵ     = 1.0,
    t_bgn = et_range[begin],
    t_end = et_range[end],
    Nt    = length(et_range),
    z_max = 0.6,
    Nz    = 41,
    P     = 7.63262 * 3600,
)

In [ ]:
# Run TPM and save the result
savepath = "TPM_Ryugu.jld2"
Astroshaper.run_TPM!(shape, et_range, sun_ryugu, thermo_params, savepath, save_range)

JLD2.jldopen(savepath, "r+") do file
    file["RYUGU_TO_J2000"] = RYUGU_TO_J2000[save_range]
end;

# Visualize the result

In [ ]:
surface_temperature(shape::Astroshaper.ShapeModel) = [facet.temps[begin] for facet in shape.facets]

function VectorVector2Matrix(v)
    m = Matrix{eltype(v[end])}(undef, length(v), 3)
    for i in eachindex(v)
        m[i, :] .= v[i]
    end
    m
end

function face2node(nodes, faces, data)
    node_data = Vector{eltype(data)}(undef, size(nodes, 1))
    
    for node_index in eachindex(node_data)
        face_indices = findall(faces.==node_index)
        if length(face_indices) == 0
            node_data[node_index] = NaN
            continue
        end
        node_data[node_index] = Statistics.mean(data[face_index[1]] for face_index in face_indices)
    end
    node_data
end

function draw(shape::Astroshaper.ShapeModel; data=nothing, r̂☉=[1,0,0.], colormap=:viridis, strokecolor=:gray20, strokewidth=1)
    nodes = VectorVector2Matrix(shape.nodes)
    faces = VectorVector2Matrix(shape.faces)

    if data === nothing
        color = :gray
    elseif data == :radius
        color = [norm(v) for v in eachrow(nodes)]
    elseif data == :temperature
        surf_temps = surface_temperature(shape)
        color = face2node(nodes, faces, surf_temps)
    elseif data == :illumination
        illumination = Float64.(isIlluminated(r̂☉, shape))
        color = face2node(nodes, faces, illumination)
    else
        color = face2node(nodes, faces, data)
    end

    scene = GLMakie.poly(nodes, faces,
        color=color, colormap=colormap,
        strokecolor=strokecolor, strokewidth=strokewidth,
        size=(1500,1500)
    )

    GLMakie.set_theme!(backgroundcolor=:white)
    display(scene)
end

In [ ]:
draw(shape; data=:temperature, colormap=:vik, strokewidth=0.05)